In [5]:
import pandas as pd
import numpy as np
import gzip
import json
import sys
import seaborn as sns
import ast
import multiprocessing
import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA
from scipy.stats import zscore
from pyod.models.mad import MAD

sys.path.append("../utils/")
from myFunctions import jsonGzipToDataframe2

Cargamos el CSV comprimido dentro de un dataframe Pandas

In [6]:
df = pd.read_csv('../datasource/user_items_chewed.csv.gz')

In [7]:
dfSteam = pd.read_csv('../datasource/steam_games_4Api.csv.gz')

In [8]:
dfUserReviews = pd.read_csv('../datasource/user_reviews_Api.csv.gz')

In [9]:
dfUserReviews.head(20)

,user_id,item_id,recommend
0,76561197970982479,1250,True
1,76561197970982479,22200,True
2,76561197970982479,43110,True
3,js41637,251610,True
4,js41637,227300,True
5,js41637,239030,True
6,evcentric,248820,True
7,evcentric,370360,True
8,evcentric,237930,True
9,evcentric,263360,True


In [ ]:
dfSteam

In [15]:
dfSteam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28701 entries, 0 to 28700
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            28701 non-null  int64  
 1   price                         27594 non-null  float64
 2   genres_indie                  28701 non-null  int64  
 3   genres_action                 28701 non-null  int64  
 4   genres_casual                 28701 non-null  int64  
 5   genres_adventure              28701 non-null  int64  
 6   genres_strategy               28701 non-null  int64  
 7   genres_simulation             28701 non-null  int64  
 8   genres_rpg                    28701 non-null  int64  
 9   genres_free_to_play           28701 non-null  int64  
 10  genres_early_access           28701 non-null  int64  
 11  genres_sports                 28701 non-null  int64  
 12  genres_massively_multiplayer  28701 non-null  int64  
 13  g

In [10]:
dfSteam = dfSteam[[
  'id',
  'price',
  'genres_indie',  
  'genres_action',  
  'genres_casual',  
  'genres_adventure',  
  'genres_strategy',  
  'genres_simulation',  
  'genres_rpg',  
  'genres_free_to_play',  
  'genres_early_access',  
  'genres_sports',  
  'genres_massively_multiplayer',  
  'genres_racing',  
  'year'
]]

In [16]:
dfSteam

,id,price,genres_indie,genres_action,genres_casual,genres_adventure,genres_strategy,genres_simulation,genres_rpg,genres_free_to_play,genres_early_access,genres_sports,genres_massively_multiplayer,genres_racing,year
0,761140,4.99,1,1,1,0,1,1,0,0,0,0,0,0,2018
1,643980,0.00,1,0,0,0,1,0,1,1,0,0,0,0,2018
2,670290,0.00,1,0,1,0,0,1,0,1,0,1,0,0,2017
3,767400,0.99,0,1,1,1,0,0,0,0,0,0,0,0,2017
4,772540,3.99,0,1,0,1,0,1,0,0,0,0,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28696,745400,1.99,1,1,1,1,0,0,0,0,0,0,0,0,2018
28697,773640,1.99,1,0,1,0,1,1,0,0,0,0,0,0,2018
28698,733530,4.99,1,0,1,0,1,0,0,0,0,0,0,0,2018
28699,610660,1.99,1,0,0,0,0,1,0,0,0,0,0,1,2018


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           int64 
 2   item_name         object
 3   items_count       int64 
 4   playtime_forever  int64 
 5   playtime_2weeks   int64 
dtypes: int64(4), object(2)
memory usage: 235.9+ MB


In [96]:
df.head(10)

,user_id,item_id,item_name,items_count,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,277,6,0
1,76561197970982479,20,Team Fortress Classic,277,0,0
2,76561197970982479,30,Day of Defeat,277,7,0
3,76561197970982479,40,Deathmatch Classic,277,0,0
4,76561197970982479,50,Half-Life: Opposing Force,277,0,0
5,76561197970982479,60,Ricochet,277,0,0
6,76561197970982479,70,Half-Life,277,0,0
7,76561197970982479,130,Half-Life: Blue Shift,277,0,0
8,76561197970982479,300,Day of Defeat: Source,277,4733,0
9,76561197970982479,240,Counter-Strike: Source,277,1853,0


In [11]:
# Remove 'playtime_2weeks' column
dfUserItems = df.drop(['playtime_2weeks'], axis = 1)

In [31]:
dfUserItems

,user_id,item_id,item_name,items_count,playtime_forever
0,76561197970982479,10,Counter-Strike,277,6
1,76561197970982479,20,Team Fortress Classic,277,0
2,76561197970982479,30,Day of Defeat,277,7
3,76561197970982479,40,Deathmatch Classic,277,0
4,76561197970982479,50,Half-Life: Opposing Force,277,0
...,...,...,...,...,...
5153204,76561198329548331,346330,BrainBread 2,7,0
5153205,76561198329548331,373330,All Is Dust,7,0
5153206,76561198329548331,388490,One Way To Die: Steam Edition,7,3
5153207,76561198329548331,521570,You Have 10 Seconds 2,7,4


In [24]:
dfSteam[dfSteam['id'] == 772540]

,id,price,genres_indie,genres_action,genres_casual,genres_adventure,genres_strategy,genres_simulation,genres_rpg,genres_free_to_play,genres_early_access,genres_sports,genres_massively_multiplayer,genres_racing,year
4,772540,3.99,0,1,0,1,0,1,0,0,0,0,0,0,2018


#### LAS CONSULTAS PARA LA API

In [41]:
def userdata(userId: str):
  # Filtrar el dataframe dfUserItems por usuario
  userItems = dfUserItems[dfUserItems['user_id'] == userId]

  # Unir el dataframe filtrado con los dfSteam que coincidan con 'item_id'
  itemsWithPrices = userItems.merge(dfSteam, left_on = 'item_id', right_on = 'id', how = 'inner')
  
  # Calcular el gasto total
  totalSpent = itemsWithPrices['price'].sum()
  
  # Calcular el porcentaje de reviews
  userRecommends = dfUserReviews.query('user_id == @userId and recommend == True').shape[0]
  per = (userRecommends * 100) / userItems.shape[0]

  # Crear el diccionario de resultados
  return {
      'Usuario': userId,
      'Dinero gastado': round(totalSpent, 2),
      '% de recomendacion': round(per, 2),
      'Cantidad de items': userItems.shape[0]
  }

In [42]:
userdata('doctr')

{'Usuario': 'doctr',
 'Dinero gastado': 6655.59,
 '% de recomendacion': 1.11,
 'Cantidad de items': 541}

In [43]:
def UserForGenre(genero: str):
  """
  Obtiene el usuario que acumula más horas jugadas y una lista de la acumulación de horas jugadas por año del juego.

  Parameters:
    genero (str): This is the genre on which the search will be based

  Returns:
    A dictionary with the following information:
      - "User with the most hours played": The ID of the user with the most hours played.
      - "Hours played": A list of dictionaries with the following information:
        - "Year": The year the game was released.
        - "Hours": The number of hours played in that year.
  """
  # Let's put together the name of the genre column to search in dfSteam
  genreSteam = f'genres_{genero}'
    
  # In 'gamesGenre' I will only have Steam games that match the genre
  gamesGenre = dfSteam[dfSteam[genreSteam] == 1].reset_index(drop = True)

  # Let's add the 'gamesGenre' columns to 'dfUserItems'. Now we have the years of the games
  # next to the items!
  dfUnion = dfUserItems.merge(gamesGenre, left_on = 'item_id', right_on = 'id', how = 'inner')
    
  # Calculate total hours played per user
  totalPlaytimeXUser = dfUnion.groupby('user_id')['playtime_forever'].sum()

  # Let's find the user with the most hours played
  userWithMostPlaytime = totalPlaytimeXUser.idxmax()

  # Now that we have found the user, let's filter it to only be left with their items
  userItems = dfUnion[dfUnion['user_id'] == userWithMostPlaytime]

  # userItemsYear keeps only the columns we need
  userItems = userItems[['year','playtime_forever']]
  
  # Let's group the DataFrame 'userItems' by year and add the column 'playtime_forever'
  userItems = userItems.groupby('year')['playtime_forever'].sum().reset_index()

  # Let's rename the column titles to fit the result dictionary
  dictionary = userItems.rename(
    columns = {'year': 'Año', 'playtime_forever': 'Horas'}).to_dict(orient='records')

  # Create the results dictionary
  return {"Usuario con más horas jugadas para el género " + genero: userWithMostPlaytime,
                "Horas jugadas por año": dictionary}

In [45]:
UserForGenre('indie')

{'Usuario con más horas jugadas para el género indie': 'REBAS_AS_F-T',
 'Horas jugadas por año': [{'Año': 1999, 'Horas': 0},
  {'Año': 2001, 'Horas': 11},
  {'Año': 2003, 'Horas': 1863},
  {'Año': 2005, 'Horas': 0},
  {'Año': 2006, 'Horas': 1673},
  {'Año': 2007, 'Horas': 1070},
  {'Año': 2008, 'Horas': 1366},
  {'Año': 2009, 'Horas': 28993},
  {'Año': 2010, 'Horas': 21487},
  {'Año': 2011, 'Horas': 100155},
  {'Año': 2012, 'Horas': 148459},
  {'Año': 2013, 'Horas': 169349},
  {'Año': 2014, 'Horas': 326927},
  {'Año': 2015, 'Horas': 751765},
  {'Año': 2016, 'Horas': 815989},
  {'Año': 2017, 'Horas': 33887}]}